In [31]:
import pandas as pd
import gzip

In [108]:
with open('tipsun30.tsv', 'rb') as fd:
    gzip_fd = gzip.GzipFile(fileobj=fd)
    data = pd.read_csv(gzip_fd,sep='\t' )

In [122]:
country = data.apply(lambda x: str(x[0]).split(',')[4], axis=1)
data['Country']=country
df = data.drop(data.columns[[0]], axis=1) 
df = df.set_index('Country')
new_rates = df.loc[:,['2017Q1 ', '2017Q2 ' ,'2017Q3 ']]

In [120]:
new_rates

,2017Q1,2017Q2,2017Q3
Country,,,
AT,5.7,5.5,:
BE,7.6 b,7.3,:
BG,6.5,6.2,:
CY,12.4,11.3,:
CZ,3.3,3.1,:
DE,3.9,3.8,:
DK,6.0,5.7,:
EE,5.7,6.5,:
EL,22.6,21.6,:
